# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [10]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')
import hvplot.pandas

In [11]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

In [12]:
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [8]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [9]:
# Define Panel Visualization Functions
def housing_units_per_year(df, var):
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    fig_housing_units = df.groupby(var).mean()
    
    housing_units_per_year_plot = fig_housing_units.hvplot.bar(x="year", y='housing_units', xlabel='Year', ylabel='Housing Units', rot=90, title='Housing Units in San Francisco from 2010 to 2016', ylim=(370000, 390000))

    return housing_units_per_year_plot
    
interact(housing_units_per_year, df=sfo_data, var='year')

def average_gross_rent(df, var1, var2):
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    avg_housing_price = df.groupby(var1).mean()

    average_gross_rent_plot = avg_housing_price.hvplot.line(x=var1, y=var2, xlabel='Year', ylabel='Gross Rent', title='Average Gross Rent in San Francisco')

    return average_gross_rent_plot

interact(average_gross_rent, df=sfo_data, var1='year', var2='gross_rent')

def average_sales_price(df, var1, var2):
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    avg_housing_price = df.groupby(var1).mean()

    average_sales_price_plot = avg_housing_price.hvplot.line(x=var1, y=var2, xlabel='Year', ylabel='Avg. Sale Price', title='Average Sale Price per Square Foot in San Francisco')

    return average_sales_price_plot

interact(average_sales_price, df=sfo_data, var1='year', var2='sale_price_sqr_foot')

def average_price_by_neighborhood(df, var1, var2, var3):
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    average_price_by_neighborhood_plot = df.hvplot.line(x=var1, y=var2, groupby=var3, xlabel='Year', ylabel='Avg. Sale Price', title='Average Sale Price per Square Foot by Neighborhood')

    return average_price_by_neighborhood_plot

interact(average_price_by_neighborhood, df=sfo_data, var1='year', var2='sale_price_sqr_foot', var3='neighborhood')

def top_most_expensive_neighborhoods(df, var1, var2):
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    avg_price_neighborhood = df.groupby(var1, axis=0).mean()
    top_10 = avg_price_neighborhood.nlargest(10, var2)

    top_most_expensive_neighborhoods_plot = top_10.hvplot.bar(x=var1, y=var2, xlabel='Neighborhood', ylabel='Sale Price per Sqr     Foot', rot=90, title = 'Top 10 Most Expesnsive Neighborhoods', ylim=(600,950))

    return top_most_expensive_neighborhoods_plot

interact(top_most_expensive_neighborhoods, df=sfo_data, var1='neighborhood', var2='sale_price_sqr_foot')

def parallel_coordinates(df, var1, var2):
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    avg_price_neighborhood = df.groupby(var1, axis=0).mean()
    top_10 = avg_price_neighborhood.nlargest(10, var2)

    parallel_coordinates_plot = px.parallel_coordinates(top_10, color=var2)

    return parallel_coordinates_plot

interact(parallel_coordinates, df=sfo_data, var1='neighborhood', var2='sale_price_sqr_foot')

def parallel_categories(df, var1, var2, var3, var4):
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    avg_price_neighborhood = df.groupby(var1, axis=0).mean()
    top_10 = avg_price_neighborhood.nlargest(10, var2)

    parallel_categories_plot = px.parallel_categories(top_10, dimensions=[top_10.index, var2, var3, var4], color=var2, color_continuous_scale=px.colors.sequential.Inferno)

    return parallel_categories_plot

interact(parallel_categories, df=sfo_data, var1='neighborhood', var2='sale_price_sqr_foot', var3='housing_units', var4='gross_rent')

def neighborhood_map(df, df1, var1, y):
    """Neighborhood Map"""
    
    # YOUR CODE HERE!
    df_neighborhood = df.groupby(var1, axis=0, as_index=False, sort=True).mean()
    df_neighborhood.rename(columns={'neighborhood': 'Neighborhood'}, inplace=True)

    left = df_neighborhood
    right = df_neighborhood_locations

    df_merge = pd.merge(left, right, how = 'inner', on =var2)

    neighborhood_map = px.scatter_mapbox(df_merge, lat=var3, lon=var4, size=var5, color=var2, zoom=4)

    return neighborhood_map


interact(neighborhood_map, sfo_data, df_neighborhood_locations, var1='neighborhood', var2='Neighborhood', var3='Lat', var4='Lon', var5='sale_price_sqr_foot')


NameError: name 'interact' is not defined

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [5]:
# YOUR CODE HERE!
housing_units_per_year_plot 
average_gross_rent_plot
average_sales_price_plot 
average_price_by_neighborhood_plot
top_most_expensive_neighborhoods_plot
parallel_coordinates_plot
parallel_categories_plot
neighborhood_map

avg_column = pn.column('## Housing Pricing Plots',
    housing_units_per_year_plot, 
    average_gross_rent_plot,
    average_sales_price_plot, 
    average_price_by_neighborhood_plot)

top_10_plots = pn.column('## Top 10 plots',
    top_most_expensive_neighborhoods_plot,
    parallel_coordinates_plot,
    parallel_categories_plot,
    neighborhood_map)

housing_price_dashboard = pn.Tabs(
    ('Housing Prices', avg_column),
    ('Top 10', top_10_plots)
    )

housing_price_dashboard

## Serve the Panel Dashboard

In [5]:
panel.servable()

NameError: name 'panel' is not defined